Step Two --- Cleaning the data

In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
raw_data = 'E:\\Cycling_Report\\data\\cycling.txt'
clean_data = 'E:\\Cycling_Report\\data\\cycling_clean.csv'
diagnostics = 'E:\\Cycling_Report\\analysis\\diagnostics.txt'

In [11]:
data = pd.read_csv(raw_data,sep = None,engine='python')
# data.head()
data.shape

(3496, 5)

In [12]:
data.info

<bound method DataFrame.info of            all_riders  rider_class stage  points stage_class
0       Tadej Pogačar  All Rounder    X1      15        flat
1       Tadej Pogačar  All Rounder    X2     219       hills
2       Tadej Pogačar  All Rounder    X3      34        flat
3       Tadej Pogačar  All Rounder    X4     264       hills
4       Tadej Pogačar  All Rounder    X6     114       hills
...               ...          ...   ...     ...         ...
3491  Fabian Lienhard    Unclassed   X17       0        flat
3492  Fabian Lienhard    Unclassed   X18       0       mount
3493  Fabian Lienhard    Unclassed   X19       0       mount
3494  Fabian Lienhard    Unclassed   X20       0       hills
3495  Fabian Lienhard    Unclassed   X21       0        flat

[3496 rows x 5 columns]>

In [13]:
data.columns.tolist()

['all_riders', 'rider_class', 'stage', 'points', 'stage_class']

In [15]:
missing_values = data.isna().sum().sort_values(ascending = False)
print("Missing Values per column :", missing_values)

Missing Values per column : all_riders     0
rider_class    0
stage          0
points         0
stage_class    0
dtype: int64


In [16]:
# Unique values
col = 'all_riders', 'rider_class', 'stage', 'points', 'stage_class'

for i in col:
    if i in data.columns:
        print(f"Unique values in {i} :")
        print(data[i].dropna().unique()[:50])

Unique values in all_riders :
['Tadej Pogačar' 'Jonas Vingegaard' 'Remco Evenepoel' 'João Almeida'
 'Primož Roglič' 'Jasper Philipsen' 'Jonathan Milan' 'Mattias Skjelmose'
 'Florian Lipowitz' 'Matteo Jorgenson' 'Wout Van Aert' 'Simon Yates'
 'Mathieu Van Der Poel' 'Lenny Martinez' 'Felix Gall' 'Carlos Rodríguez'
 'Enric Mas' 'Tim Merlier' 'Adam Yates' 'Tobias Halland Johannessen'
 'Kaden Groves' 'Santiago Buitrago' 'Romain Grégoire' 'Thymen Arensman'
 'Filippo Ganna' 'Geraint Thomas' 'Biniam Girmay' 'Thibau Nys'
 'Arnaud De Lie' 'Jordi Meeus' 'Aleksandr Vlasov' 'Dylan Groenewegen'
 "Ben O'Connor" 'Oscar Onley' 'Sepp Kuss' 'Xandro Meurisse'
 'Gianni Vermeersch' 'Emiel Verstrynge' 'Amaury Capiot' 'Ewen Costiou'
 'Arnaud Démare' 'Raúl García Pierna' 'Cristián Rodríguez'
 'Kévin Vauquelin' 'Clément Venturini' 'Phil Bauhaus' 'Jack Haig'
 'Matej Mohorič' 'Fred Wright' 'Alex Aranburu']
Unique values in rider_class :
['All Rounder' 'Sprinter' 'Climber' 'Unclassed']
Unique values in stage :
['X

Clean 'point' column

In [17]:
if 'points' in data.columns:
    not_numeric = ~data['points'].apply(lambda a: pd.to_numeric(a, errors = 'coerce')).notna()
    print("Example non-numeric 'points' entries:\n", data.loc[not_numeric,'points'].unique()[:20])

    data['points'] = pd.to_numeric(data['points'].astype(str).str.replace(',',''),errors = 'coerce')
    print("After coerce, missing values in points :", data['points'].isna().sum())

else:
    raise KeyError("No 'points' column found.")

Example non-numeric 'points' entries:
 []
After coerce, missing values in points : 0


Normalizing catagorical columns

In [19]:
def clean_s(s):
    if pd.isna(s):
        return s

    s = str(s).strip()
    s = s.replace('-', ' ').replace('_', ' ')
    s = " ".join(s.split())
    s = s.title()

    return s

catego = ['rider_class','stage_class']
for cate in catego:
    if cate in data.columns:
        data[cate] = data[cate].apply(clean_s)
        data[cate] = data[cate].astype('category')

        print(cate, "levels :", data[cate].cat.categories.tolist())

    else:
        print(f'Alert : {cate} not found in dataset.')

rider_class levels : ['All Rounder', 'Climber', 'Sprinter', 'Unclassed']
stage_class levels : ['Flat', 'Hills', 'Mount']


In [20]:
mapp = {'Allrounder':'All Rounder', 'All-Rounder':'All Rounder', 'Allround': 'All Rounder'}
data['rider_class'] = data['rider_class'].replace(mapp)
data['rider_class'] = data['rider_class'].astype('category')

Summary Variables

In [21]:
# Total points per rider assuming rider id exists

if 'rider_id' in data.columns:
    T_Points = data.groupby('rider_id', dropna = False)['points'].sum().reset_index().rename(columns = {'points' : 'T_points'})
    data = data.merge(T_Points, on = 'rider_id', how = 'left')

    avg_stage = data.groupby(['rider_id','stage_class'],dropna = False)['points'].mean().unstack(fill_value= np.nan)
    avg_stage.columns = [f'avg_stage_points_{str(c)}' for c in avg_stage.columns]
    data = data.merge(avg_stage.reset_index(), on = 'rider_id', how = 'left')

else:
    print("No rider_id : calculate total by rider_name or skip totals per rider.")

No rider_id : calculate total by rider_name or skip totals per rider.


calculate average points per stage_class

In [22]:
avg_stage_class = data.groupby('stage_class')['points'].agg(['count', 'mean', 'median', 'std']).reset_index()
print(avg_stage_class)

# saving the summary

avg_stage_class.to_csv('E:\\Cycling_Report\\analysis\\avg_points_stage_class.csv', index = False)

  stage_class  count       mean  median        std
0        Flat   1104  11.794384     0.0  33.219268
1       Hills   1472  12.520380     0.0  36.130357
2       Mount    920  12.878261     0.0  39.906588


C:\Users\luxma\AppData\Local\Temp\ipykernel_8508\3161914952.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  avg_stage_class = data.groupby('stage_class')['points'].agg(['count', 'mean', 'median', 'std']).reset_index()


Handling Missing Values

In [23]:
# missing fraction

missing_fraction_points = data['points'].isna().mean()
print("Missing fraction in points :", missing_fraction_points)

if missing_fraction_points < 0.05:
    # drop rows with missing points

    data = data.dropna(subset = ['points']).reset_index(drop = True)
    act = "rows with missing points (<5%) are dropped."

else:
    # median imputation by rider_class
    data['points'] = data.groupby('rider_class')['points'].apply(lambda e: e.fillna(e.median()))
    # remaining Nans ( filling with overall median if rider_class missing)
    data['points'] = data['points'].fillna(data['points'].median())
    act = "missing points using rider_class median are imputed, then overall median"

print("Now Missing Points :", data['points'].isna().sum(),", action :", act)

Missing fraction in points : 0.0
Now Missing Points : 0 , action : rows with missing points (<5%) are dropped.


Catagorical Missing Values

In [24]:
for c in ['rider_class','stage_class']:
    if c in data.columns:
        data[c] = data[c].cat.add_categories(['Unknown']).fillna('Unknown')

Categorical lavel consistency

In [25]:
for c in ['rider_class','stage_class']:
    if c in data.columns:
        print(c, "levels now :", data[c].astype(str).unique())

rider_class levels now : ['All Rounder' 'Sprinter' 'Climber' 'Unclassed']
stage_class levels now : ['Flat' 'Hills' 'Mount']


Saving cleaned CSV and Diagnostics

In [27]:
# Cleaned file
os.makedirs('E:\Cycling_Report\data',exist_ok = True)
data.to_csv(clean_data, index = False)

# Diagnostics

with open(diagnostics, 'w') as f:
    f.write("Cleaning diagnostics\n")
    f.write(f"Original file : {raw_data}\n")
    f.write(f"Cleaned file : {clean_data}\n")
    f.write(f"Action on points : {act}\n\n")
    f.write("Missing values after cleaning:\n")
    f.write(str(data.isna().sum()))

print("Cleaned CSV and diagnostics are saved.")

Cleaned CSV and diagnostics are saved.


<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\luxma\AppData\Local\Temp\ipykernel_8508\3264738547.py:2: SyntaxWarning: invalid escape sequence '\C'
  os.makedirs('E:\Cycling_Report\data',exist_ok = True)


Verification

In [28]:
data2 = pd.read_csv(clean_data)
print(data2.info())
print(data2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3496 entries, 0 to 3495
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   all_riders   3496 non-null   object
 1   rider_class  3496 non-null   object
 2   stage        3496 non-null   object
 3   points       3496 non-null   int64 
 4   stage_class  3496 non-null   object
dtypes: int64(1), object(4)
memory usage: 136.7+ KB
None
      all_riders  rider_class stage  points stage_class
0  Tadej Pogačar  All Rounder    X1      15        Flat
1  Tadej Pogačar  All Rounder    X2     219       Hills
2  Tadej Pogačar  All Rounder    X3      34        Flat
3  Tadej Pogačar  All Rounder    X4     264       Hills
4  Tadej Pogačar  All Rounder    X6     114       Hills
